## Intelligent Business Operations Concierge Agent (IBOCA)

This project combines **customer support automation** with **workflow improvement** and **data analysis** into a single, cohesive multi-agent system.

A **Business Operation Concierge** will be created. This single-track agent will act as an intelligent front door for internal business operations, triaging requests and orchestrating specialized sub-agents.

In [1]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [2]:
root_agent = Agent(
    name="helpful_assistant",
    model="gemini-2.5-flash-lite",
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

In [3]:
from kaggle_secrets import UserSecretsClient
import os

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Setup complete (Google AI Studio).")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Setup complete (Google AI Studio).


In [ ]:
# Display folder structure
print("Project Structure:")
print("""
business-ops-agent/
├── agents/
│   ├── orchestrator.py
│   ├── support_agent.py
│   ├── workflow_agent.py
│   └── data_agent.py
├── tools/
│   ├── __init__.py
│   ├── hr_tools.py
│   ├── it_tools.py
│   ├── data_tools.py
│   └── email_tools.py
├── memory/
│   ├── session_manager.py
│   └── memory_bank.py
├── evaluation/
│   └── test_suite.py
├── app.py
├── requirements.txt
├── Dockerfile
└── README.md
""")                  # Documentation

### 1. Main Application with Multi-Agent System (app.py)

In [ ]:
# Cell 1: Install all dependencies
!pip install -q langchain langchain-community langchain-openai google-search-results fastapi uvicorn pandas openai

print("✅ All dependencies installed!")

In [ ]:
# ============================================
# CELL 1: CREATE ALL FILES AND FOLDERS
# ============================================
import os

# Create directories
directories = ["agents", "tools", "memory", "evaluation"]
for dir_name in directories:
    os.makedirs(dir_name, exist_ok=True)
    print(f"📁 Created folder: {dir_name}/")

# Create __init__.py files (makes them importable as modules)
for dir_name in directories:
    init_file = os.path.join(dir_name, "__init__.py")
    with open(init_file, "w") as f:
        f.write('"""Package initialization"""\n')
    print(f"📄 Created: {init_file}")

print("\n✅ All folders created!")

In [ ]:
# ============================================
# COMPLETE SETUP CELL - RUN THIS FIRST!
# ============================================
import os
import sys
import shutil

print("🧹 Cleaning up old files...")
# Remove old folders if they exist
for folder in ["agents", "tools", "memory", "evaluation"]:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"  Removed: {folder}/")

print("\n📁 Creating new folder structure...")
# Create fresh folders
folders = ["agents", "tools", "memory", "evaluation"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)
    # Create __init__.py to make it a proper Python package
    with open(os.path.join(folder, "__init__.py"), "w") as f:
        f.write('"""Package initialization"""\n')
    print(f"  Created: {folder}/")

print("\n📄 Creating Python files...")

# 1. Create agents/orchestrator.py
orchestrator_code = '''"""
Orchestrator Agent
"""
import json

class OrchestratorAgent:
    """Main orchestrator implementing A2A protocol patterns"""
    
    def __init__(self, llm=None, tools=None, support_agent=None, 
                 workflow_agent=None, data_agent=None, memory_bank=None):
        self.llm = llm
        self.tools = tools or []
        self.support_agent = support_agent
        self.workflow_agent = workflow_agent
        self.data_agent = data_agent
        self.memory_bank = memory_bank
        
    async def coordinate_agents(self, task: str):
        """Coordinate between multiple agents (A2A Protocol)"""
        return {
            "status": "success",
            "task": task,
            "agents_involved": ["support", "workflow", "data"],
            "result": f"Processed: {task} using multiple agents"
        }
    
    async def use_tools(self, query: str, tool_types=None):
        """Use available tools based on query"""
        if tool_types:
            return f"Using tools of type {tool_types} for query: {query}"
        return f"Using all available tools for query: {query}"

# For testing
if __name__ == "__main__":
    agent = OrchestratorAgent()
    print("OrchestratorAgent created successfully!")
'''

with open("agents/orchestrator.py", "w") as f:
    f.write(orchestrator_code)
print("  ✅ agents/orchestrator.py")

# 2. Create agents/support_agent.py
support_agent_code = '''"""
Support Specialist Agent
"""
class SupportSpecialist:
    """Support specialist agent for handling inquiries"""
    
    def __init__(self, llm=None):
        self.llm = llm
    
    async def answer_question(self, question: str) -> str:
        """Answer support questions"""
        return f"I can help with: {question}. Based on our knowledge base, the solution is..."
    
    async def check_knowledge_base(self, query: str) -> str:
        """Check knowledge base for answers"""
        answers = {
            "password reset": "Go to portal.example.com/password-reset",
            "vpn issues": "Try restarting the VPN client or contact IT",
            "software installation": "Download from company portal or request admin rights"
        }
        for key in answers:
            if key in query.lower():
                return answers[key]
        return "I couldn't find a specific answer. Let me connect you with a human agent."

# For testing
if __name__ == "__main__":
    agent = SupportSpecialist()
    print("SupportSpecialist created successfully!")
'''

with open("agents/support_agent.py", "w") as f:
    f.write(support_agent_code)
print("  ✅ agents/support_agent.py")

# 3. Create agents/workflow_agent.py
workflow_agent_code = '''"""
Workflow Automation Agent
"""
class WorkflowAgent:
    """Workflow automation agent for sequential processes"""
    
    def __init__(self, llm=None):
        self.llm = llm
    
    async def execute_workflow(self, workflow_name: str) -> str:
        """Execute a workflow sequentially"""
        steps = {
            "leave_request": ["Submit form", "Manager approval", "HR processing", "Confirmation"],
            "expense_report": ["Collect receipts", "Fill form", "Submit", "Finance review", "Reimbursement"],
            "onboarding": ["Paperwork", "IT setup", "Training", "Team introduction"]
        }
        
        if workflow_name in steps:
            workflow_steps = steps[workflow_name]
            result = f"Executing '{workflow_name}' workflow:\\n"
            for i, step in enumerate(workflow_steps, 1):
                result += f"  Step {i}: {step} ✓\\n"
            return result + "✅ Workflow completed!"
        return f"Workflow '{workflow_name}' not found."

# For testing
if __name__ == "__main__":
    agent = WorkflowAgent()
    print("WorkflowAgent created successfully!")
'''

with open("agents/workflow_agent.py", "w") as f:
    f.write(workflow_agent_code)
print("  ✅ agents/workflow_agent.py")

# 4. Create agents/data_agent.py
data_agent_code = '''"""
Data Analysis Agent
"""
import asyncio

class DataAnalysisAgent:
    """Data analysis agent for parallel processing"""
    
    def __init__(self, llm=None):
        self.llm = llm
    
    async def analyze_trends(self, period="monthly") -> str:
        """Analyze data trends"""
        await asyncio.sleep(0.1)  # Simulate processing
        return f"{period.capitalize()} trends: Support tickets ↓15%, Resolution time ↓20%, Satisfaction ↑10%"
    
    async def generate_report(self, report_type: str) -> str:
        """Generate a report"""
        await asyncio.sleep(0.1)
        reports = {
            "weekly": "Weekly Report: 150 tickets, 85% resolution rate",
            "monthly": "Monthly Report: 600 tickets, Avg response: 2.1h",
            "quarterly": "Q1 Report: 2400 tickets, CSAT: 4.5/5.0"
        }
        return reports.get(report_type, f"Standard report for {report_type}")
    
    async def calculate_metrics(self) -> str:
        """Calculate business metrics"""
        await asyncio.sleep(0.1)
        return "Current Metrics: Active tickets: 42, SLA compliance: 92%, Agent utilization: 78%"

# For testing
if __name__ == "__main__":
    agent = DataAnalysisAgent()
    print("DataAnalysisAgent created successfully!")
'''

with open("agents/data_agent.py", "w") as f:
    f.write(data_agent_code)
print("  ✅ agents/data_agent.py")

# 5. Create tools/hr_tools.py
hr_tools_code = '''"""
HR Tools Module
"""
class HRToolkit:
    """Collection of HR-related tools"""
    
    def get_tools(self):
        return [LeaveBalanceTool(), PolicyLookupTool(), EmployeeDirectoryTool()]

class LeaveBalanceTool:
    """Custom tool for checking leave balance"""
    
    name = "check_leave_balance"
    description = "Check an employee's remaining leave balance"
    tool_type = "hr_tools"
    
    def run(self, employee_id: str, leave_type: str = None) -> str:
        """Execute the tool"""
        balances = {
            "EMP001": {"vacation": 15, "sick": 10, "personal": 5},
            "EMP002": {"vacation": 20, "sick": 8, "personal": 3},
            "EMP003": {"vacation": 12, "sick": 12, "personal": 8}
        }
        
        if employee_id not in balances:
            return f"Employee {employee_id} not found"
        
        if leave_type:
            if leave_type in balances[employee_id]:
                balance = balances[employee_id][leave_type]
                return f"Employee {employee_id} has {balance} days of {leave_type} leave"
            return f"Leave type '{leave_type}' not found"
        
        # Return all balances
        return f"Leave balances for {employee_id}: {balances[employee_id]}"

class PolicyLookupTool:
    """Tool to look up HR policies"""
    
    name = "lookup_policy"
    description = "Look up HR policies by topic"
    tool_type = "hr_tools"
    
    def run(self, policy_topic: str) -> str:
        policies = {
            "remote_work": "Employees may work remotely 3 days/week with manager approval.",
            "leave_request": "Submit leave requests 2 weeks in advance via HR portal.",
            "expenses": "Expense reports must be submitted within 30 days with receipts.",
            "overtime": "Overtime requires pre-approval and is paid at 1.5x rate."
        }
        return policies.get(policy_topic.lower(), f"No policy found for: {policy_topic}")

class EmployeeDirectoryTool:
    """Employee directory tool"""
    
    name = "employee_directory"
    description = "Look up employee information"
    tool_type = "hr_tools"
    
    def run(self, employee_id: str = None, department: str = None) -> str:
        employees = [
            {"id": "EMP001", "name": "John Doe", "department": "Engineering", "role": "Developer"},
            {"id": "EMP002", "name": "Jane Smith", "department": "Marketing", "role": "Manager"},
            {"id": "EMP003", "name": "Bob Johnson", "department": "HR", "role": "Specialist"}
        ]
        
        if employee_id:
            for emp in employees:
                if emp["id"] == employee_id:
                    return f"Employee: {emp['name']}, Dept: {emp['department']}, Role: {emp['role']}"
            return f"Employee {employee_id} not found"
        
        if department:
            dept_employees = [e for e in employees if e["department"].lower() == department.lower()]
            if dept_employees:
                return f"Employees in {department}: {[e['name'] for e in dept_employees]}"
            return f"No employees found in {department}"
        
        return f"Total employees: {len(employees)}"

# For testing
if __name__ == "__main__":
    toolkit = HRToolkit()
    print(f"HR Toolkit created with {len(toolkit.get_tools())} tools")
'''

with open("tools/hr_tools.py", "w") as f:
    f.write(hr_tools_code)
print("  ✅ tools/hr_tools.py")

# 6. Create memory/session_manager.py
session_manager_code = '''"""
Session Manager
"""
from datetime import datetime

class SessionManager:
    """Manage user sessions"""
    
    def __init__(self):
        self.sessions = {}
    
    def create_session(self, user_id: str) -> str:
        """Create a new session"""
        session_id = f"{user_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.sessions[session_id] = {
            "user_id": user_id,
            "created_at": datetime.now(),
            "data": {},
            "history": []
        }
        return session_id
    
    def update_session(self, session_id: str, data: dict) -> bool:
        """Update session data"""
        if session_id in self.sessions:
            self.sessions[session_id]["data"].update(data)
            self.sessions[session_id]["history"].append({
                "timestamp": datetime.now(),
                "action": "update",
                "data": data
            })
            return True
        return False
    
    def get_session(self, session_id: str) -> dict:
        """Get session data"""
        return self.sessions.get(session_id, {})
    
    def end_session(self, session_id: str) -> bool:
        """End a session"""
        if session_id in self.sessions:
            del self.sessions[session_id]
            return True
        return False

# For testing
if __name__ == "__main__":
    manager = SessionManager()
    session_id = manager.create_session("EMP001")
    print(f"Session created: {session_id}")
'''

with open("memory/session_manager.py", "w") as f:
    f.write(session_manager_code)
print("  ✅ memory/session_manager.py")

# 7. Create memory/memory_bank.py
memory_bank_code = '''"""
Memory Bank for long-term storage
"""
from datetime import datetime, timedelta
from collections import defaultdict

class MemoryBank:
    """Long-term memory storage with context compaction"""
    
    def __init__(self, max_entries_per_user: int = 50):
        self.memory_store = defaultdict(list)
        self.max_entries = max_entries_per_user
    
    def store(self, user_id: str, key: str, value: str, metadata: dict = None) -> str:
        """Store information in long-term memory"""
        entry = {
            "id": f"{user_id}_{key}_{datetime.now().timestamp()}",
            "key": key,
            "value": value,
            "timestamp": datetime.now(),
            "metadata": metadata or {},
            "access_count": 0
        }
        
        self.memory_store[user_id].append(entry)
        
        # Apply context compaction if needed
        if len(self.memory_store[user_id]) > self.max_entries:
            self._compact_context(user_id)
        
        return entry["id"]
    
    def recall(self, user_id: str, key: str = None, limit: int = 5) -> list:
        """Recall information from memory"""
        if user_id not in self.memory_store:
            return []
        
        memories = self.memory_store[user_id]
        
        # Filter by key if specified
        if key:
            relevant = [m for m in memories if key in m["key"]]
        else:
            relevant = memories
        
        # Sort by recency
        relevant.sort(key=lambda x: x["timestamp"], reverse=True)
        
        # Update access count
        for memory in relevant[:limit]:
            memory["access_count"] += 1
        
        return [
            {
                "key": m["key"],
                "value": m["value"],
                "age": (datetime.now() - m["timestamp"]).days,
                "accessed": m["access_count"]
            }
            for m in relevant[:limit]
        ]
    
    def _compact_context(self, user_id: str):
        """Context compaction: remove least accessed memories"""
        memories = self.memory_store[user_id]
        
        # Sort by access count (least accessed first)
        memories.sort(key=lambda x: x["access_count"])
        
        # Keep only the most recent/accessed memories
        keep_count = self.max_entries // 2
        self.memory_store[user_id] = memories[-keep_count:] if len(memories) > keep_count else memories
    
    def get_stats(self, user_id: str) -> dict:
        """Get memory statistics for a user"""
        if user_id not in self.memory_store:
            return {"total_memories": 0, "last_access": None}
        
        memories = self.memory_store[user_id]
        if not memories:
            return {"total_memories": 0, "last_access": None}
        
        return {
            "total_memories": len(memories),
            "last_access": max(m["timestamp"] for m in memories).isoformat(),
            "most_accessed": max(memories, key=lambda x: x["access_count"])["key"]
        }

# For testing
if __name__ == "__main__":
    memory = MemoryBank()
    memory.store("EMP001", "hr_question", "Asked about vacation policy")
    print("MemoryBank created successfully!")
'''

with open("memory/memory_bank.py", "w") as f:
    f.write(memory_bank_code)
print("  ✅ memory/memory_bank.py")

# 8. Create evaluation/test_suite.py
test_suite_code = '''"""
Agent Evaluation Suite
"""
from datetime import datetime

class AgentEvaluator:
    """Evaluate agent performance"""
    
    def __init__(self, agent=None):
        self.agent = agent
        self.test_cases = [
            {"input": "What's my vacation balance?", "expected": "hr_inquiry", "category": "HR"},
            {"input": "My laptop won't turn on", "expected": "it_support", "category": "IT"},
            {"input": "Show last month's metrics", "expected": "data_analysis", "category": "Data"},
            {"input": "How do I submit expenses?", "expected": "workflow", "category": "Workflow"}
        ]
    
    async def evaluate_intent_classification(self) -> dict:
        """Evaluate intent classification accuracy"""
        # Simulate evaluation
        correct = 3
        total = len(self.test_cases)
        
        return {
            "accuracy": correct / total,
            "correct_count": correct,
            "total_tests": total,
            "details": [{"test": tc["input"], "status": "PASS" if i < 3 else "FAIL"} 
                       for i, tc in enumerate(self.test_cases)]
        }
    
    async def evaluate_tool_selection(self) -> dict:
        """Evaluate tool selection accuracy"""
        return {
            "tool_selection_accuracy": 0.85,
            "correct_selections": 17,
            "total_tests": 20,
            "details": "Tools correctly selected in 85% of cases"
        }
    
    async def run_complete_evaluation(self) -> dict:
        """Run all evaluations"""
        intent_results = await self.evaluate_intent_classification()
        tool_results = await self.evaluate_tool_selection()
        
        overall_score = (intent_results["accuracy"] * 0.6 + 
                        tool_results["tool_selection_accuracy"] * 0.4)
        
        return {
            "overall_score": overall_score,
            "intent_classification": intent_results,
            "tool_selection": tool_results,
            "passing_threshold": 0.7,
            "passed": overall_score >= 0.7,
            "evaluation_date": datetime.now().isoformat(),
            "agent_version": "1.0.0"
        }

# For testing
if __name__ == "__main__":
    evaluator = AgentEvaluator()
    print("AgentEvaluator created successfully!")
'''

with open("evaluation/test_suite.py", "w") as f:
    f.write(test_suite_code)
print("  ✅ evaluation/test_suite.py")

# Create requirements.txt
requirements = '''langchain>=0.1.0
langchain-community>=0.0.10
langchain-openai>=0.0.2
openai>=1.3.0
google-search-results>=2.4.2
fastapi>=0.104.1
uvicorn>=0.24.0
pandas>=2.1.3
numpy>=1.24.3
requests>=2.31.0
python-dotenv>=1.0.0
'''

with open("requirements.txt", "w") as f:
    f.write(requirements)
print("  ✅ requirements.txt")

# Create app.py stub
app_code = '''"""
Main Application - Business Operations Concierge Agent
"""
print("🚀 Business Operations Concierge Agent")
print("✅ All modules are ready to import!")
'''

with open("app.py", "w") as f:
    f.write(app_code)
print("  ✅ app.py")

print("\n" + "="*50)
print("🎉 SETUP COMPLETE!")
print("="*50)
print("\nNow you can safely run:")
print("from agents.orchestrator import OrchestratorAgent")
print("from agents.support_agent import SupportSpecialist")
print("from agents.workflow_agent import WorkflowAgent")
print("from agents.data_agent import DataAnalysisAgent")
print("from tools.hr_tools import HRToolkit")
print("from memory.session_manager import SessionManager")
print("from memory.memory_bank import MemoryBank")
print("from evaluation.test_suite import AgentEvaluator")
print("\n✅ All imports will work!")

In [ ]:
# ============================================
# CREATE tools/it_tools.py
# ============================================
import os

it_tools_code = '''"""
IT Tools Module
"""
import random
from datetime import datetime

class ITToolkit:
    """Collection of IT-related tools"""
    
    def get_tools(self):
        return [
            CreateTicketTool(),
            SystemStatusTool(),
            PasswordResetTool(),
            SoftwareInstallTool()
        ]

class CreateTicketTool:
    """Tool for creating IT support tickets"""
    
    name = "create_ticket"
    description = "Create an IT support ticket"
    tool_type = "it_tools"
    
    def run(self, description: str, priority: str = "medium", category: str = None) -> dict:
        """Create a support ticket"""
        ticket_id = f"TICKET-{datetime.now().strftime('%Y%m%d')}-{random.randint(1000, 9999)}"
        
        priorities = {"low": "Low", "medium": "Medium", "high": "High", "critical": "Critical"}
        priority_display = priorities.get(priority.lower(), "Medium")
        
        categories = ["hardware", "software", "network", "account", "other"]
        if category and category.lower() in categories:
            assigned_category = category.lower()
        else:
            # Auto-detect category from description
            desc_lower = description.lower()
            if any(word in desc_lower for word in ["laptop", "computer", "printer", "hardware"]):
                assigned_category = "hardware"
            elif any(word in desc_lower for word in ["software", "install", "update", "program"]):
                assigned_category = "software"
            elif any(word in desc_lower for word in ["wifi", "network", "internet", "vpn"]):
                assigned_category = "network"
            elif any(word in desc_lower for word in ["password", "login", "account", "access"]):
                assigned_category = "account"
            else:
                assigned_category = "other"
        
        return {
            "ticket_id": ticket_id,
            "status": "created",
            "description": description,
            "priority": priority_display,
            "category": assigned_category,
            "created_at": datetime.now().isoformat(),
            "estimated_resolution": "24-48 hours" if priority != "critical" else "4-8 hours",
            "message": f"Ticket {ticket_id} created successfully. An IT specialist will contact you soon."
        }

class SystemStatusTool:
    """Tool for checking system status"""
    
    name = "check_system_status"
    description = "Check the status of various IT systems"
    tool_type = "it_tools"
    
    def run(self, system: str = None) -> dict:
        """Check system status"""
        systems_status = {
            "email": {"status": "operational", "uptime": "99.9%", "last_incident": "7 days ago"},
            "vpn": {"status": "operational", "uptime": "99.5%", "last_incident": "2 days ago"},
            "file_server": {"status": "degraded", "uptime": "95.2%", "last_incident": "Today 09:00"},
            "website": {"status": "operational", "uptime": "99.8%", "last_incident": "14 days ago"},
            "database": {"status": "operational", "uptime": "99.95%", "last_incident": "30 days ago"}
        }
        
        if system:
            if system.lower() in systems_status:
                return {
                    "system": system,
                    **systems_status[system.lower()],
                    "checked_at": datetime.now().isoformat()
                }
            else:
                return {
                    "error": f"System '{system}' not found",
                    "available_systems": list(systems_status.keys())
                }
        
        # Return all systems status
        return {
            "overall_status": "operational",
            "systems": systems_status,
            "summary": f"{len([s for s in systems_status.values() if s['status'] == 'operational'])}/{len(systems_status)} systems operational",
            "checked_at": datetime.now().isoformat()
        }

class PasswordResetTool:
    """Tool for password reset operations"""
    
    name = "reset_password"
    description = "Reset user password or send reset link"
    tool_type = "it_tools"
    
    def run(self, username: str, method: str = "email") -> dict:
        """Reset password for a user"""
        methods = {
            "email": "Password reset link sent to registered email",
            "sms": "Password reset code sent via SMS",
            "automated": "Password has been automatically reset. Check email for new password.",
            "manual": "Password reset request logged. IT staff will contact you."
        }
        
        if method not in methods:
            method = "email"
        
        reset_id = f"RESET-{datetime.now().strftime('%H%M%S')}-{random.randint(100, 999)}"
        
        return {
            "reset_id": reset_id,
            "username": username,
            "method": method,
            "status": "initiated",
            "message": methods[method],
            "instructions": "If you don't receive the reset within 10 minutes, contact IT support.",
            "timestamp": datetime.now().isoformat()
        }

class SoftwareInstallTool:
    """Tool for software installation requests"""
    
    name = "request_software_install"
    description = "Request software installation or license"
    tool_type = "it_tools"
    
    def run(self, software_name: str, version: str = "latest", reason: str = None) -> dict:
        """Request software installation"""
        software_catalog = {
            "microsoft office": {"license_type": "enterprise", "approval_required": False},
            "adobe creative cloud": {"license_type": "subscription", "approval_required": True},
            "visual studio code": {"license_type": "free", "approval_required": False},
            "slack": {"license_type": "enterprise", "approval_required": False},
            "zoom": {"license_type": "enterprise", "approval_required": False},
            "specialized_tool": {"license_type": "individual", "approval_required": True}
        }
        
        software_lower = software_name.lower()
        
        if software_lower in software_catalog:
            software_info = software_catalog[software_lower]
        else:
            software_info = {"license_type": "unknown", "approval_required": True}
        
        request_id = f"SOFTWARE-{datetime.now().strftime('%Y%m%d')}-{random.randint(100, 999)}"
        
        approval_message = ""
        if software_info["approval_required"]:
            approval_message = "This software requires manager approval. Request has been forwarded."
        else:
            approval_message = "Software can be installed immediately. Check your email for installation link."
        
        return {
            "request_id": request_id,
            "software": software_name,
            "version": version,
            "license_type": software_info["license_type"],
            "approval_required": software_info["approval_required"],
            "status": "submitted",
            "message": approval_message,
            "reason": reason or "Not specified",
            "estimated_time": "24 hours" if software_info["approval_required"] else "2 hours",
            "submitted_at": datetime.now().isoformat()
        }

# For testing
if __name__ == "__main__":
    toolkit = ITToolkit()
    print(f"IT Toolkit created with {len(toolkit.get_tools())} tools:")
    for tool in toolkit.get_tools():
        print(f"  - {tool.name}: {tool.description}")
'''

# Create the tools directory if it doesn't exist
os.makedirs("tools", exist_ok=True)

# Create __init__.py if it doesn't exist
if not os.path.exists("tools/__init__.py"):
    with open("tools/__init__.py", "w") as f:
        f.write('"""Tools package"""')

# Write the it_tools.py file
with open("tools/it_tools.py", "w") as f:
    f.write(it_tools_code)

print("✅ Created: tools/it_tools.py")
print("\n📋 Available IT Tools:")
print("1. CreateTicketTool - Create IT support tickets")
print("2. SystemStatusTool - Check system status")
print("3. PasswordResetTool - Reset user passwords")
print("4. SoftwareInstallTool - Request software installation")

In [ ]:
import asyncio
import threading
from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.get("/")
def home():
    return {"message": "Business Ops Agent API Running"}

@app.get("/health")
def health():
    return {"status": "healthy"}

def run_server():
    """Run the server in a separate thread"""
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

# Start server in background thread
print("🚀 Starting server on http://localhost:8000")
print("📋 Available endpoints:")
print("  - GET /          - API homepage")
print("  - GET /health    - Health check")
print("  - POST /process  - Process requests")

# Start the server in a background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give server time to start
import time
time.sleep(2)

print("\n✅ Server is running in the background!")
print("💡 To stop: Kernel → Interrupt or Restart")

In [ ]:
"""
Intelligent Business Operations Concierge Agent
Demonstrates: Multi-agent system, Tools, Memory, Observability, Deployment
"""

import logging
from typing import Dict, Any, List
from datetime import datetime
import asyncio

# Import agent framework (using LangChain as example)
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.tools import GoogleSearchRun
from langchain_openai import ChatOpenAI

# Import custom modules
from agents.orchestrator import OrchestratorAgent
from agents.support_agent import SupportSpecialist
from agents.workflow_agent import WorkflowAgent
from agents.data_agent import DataAnalysisAgent
from tools.hr_tools import HRToolkit
from tools.it_tools import ITToolkit
from memory.session_manager import SessionManager
from memory.memory_bank import MemoryBank

# Configure logging for observability
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('agent_operations.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class BusinessOperationsConcierge:
    """Main orchestrator class demonstrating multi-agent system"""
    
    def __init__(self, user_id: str):
        # Initialize memory components
        self.session_manager = SessionManager()
        self.memory_bank = MemoryBank()
        
        # Create user session
        self.session_id = self.session_manager.create_session(user_id)
        
        # Initialize LLM
        self.llm = ChatOpenAI(
            model="gpt-4",
            temperature=0.3,
            streaming=True
        )
        
        # Initialize specialist agents (parallel capability)
        self.support_agent = SupportSpecialist(self.llm)
        self.workflow_agent = WorkflowAgent(self.llm)
        self.data_agent = DataAnalysisAgent(self.llm)
        
        # Initialize toolkits
        self.hr_tools = HRToolkit().get_tools()
        self.it_tools = ITToolkit().get_tools()
        self.google_search = GoogleSearchRun()
        
        # Build orchestrator agent with all tools
        all_tools = self.hr_tools + self.it_tools + [self.google_search]
        
        self.orchestrator = OrchestratorAgent(
            llm=self.llm,
            tools=all_tools,
            support_agent=self.support_agent,
            workflow_agent=self.workflow_agent,
            data_agent=self.data_agent,
            memory_bank=self.memory_bank
        )
        
        logger.info(f"Initialized BusinessOperationsConcierge for user {user_id}")
    
    async def process_request(self, user_input: str) -> Dict[str, Any]:
        """Process user request using multi-agent system"""
        
        # Log the incoming request
        logger.info(f"Processing request: {user_input}")
        
        try:
            # Classify intent and route to appropriate agent
            intent = await self._classify_intent(user_input)
            logger.info(f"Classified intent: {intent}")
            
            # Update session memory
            self.session_manager.update_session(
                self.session_id,
                {"last_input": user_input, "intent": intent, "timestamp": datetime.now()}
            )
            
            # Route to appropriate agent based on intent
            if intent == "it_support":
                response = await self._handle_it_support(user_input)
            elif intent == "hr_inquiry":
                response = await self._handle_hr_inquiry(user_input)
            elif intent == "data_analysis":
                # Run data analysis in parallel with other tasks if needed
                response = await self._handle_data_request(user_input)
            elif intent == "workflow":
                response = await self._handle_workflow(user_input)
            else:
                response = await self._handle_general_support(user_input)
            
            # Store in long-term memory if important
            if intent in ["hr_inquiry", "it_support"]:
                self.memory_bank.store(
                    user_id=self.session_id,
                    key=intent,
                    value=user_input[:100],  # Store snippet
                    metadata={"timestamp": datetime.now()}
                )
            
            return {
                "response": response,
                "intent": intent,
                "session_id": self.session_id,
                "timestamp": datetime.now().isoformat()
            }
            
        except Exception as e:
            logger.error(f"Error processing request: {e}", exc_info=True)
            return {"error": str(e), "session_id": self.session_id}
    
    async def _classify_intent(self, user_input: str) -> str:
        """Classify user intent using LLM"""
        prompt = f"""
        Classify the following user request into one of these categories:
        - it_support: Technical issues, password resets, software problems
        - hr_inquiry: Leave balance, benefits, policies, payroll
        - data_analysis: Reports, analytics, trends, metrics
        - workflow: Process automation, approvals, form submissions
        - general: Questions, information requests
        
        User request: "{user_input}"
        
        Return only the category name.
        """
        
        response = await self.llm.ainvoke(prompt)
        return response.content.strip().lower()
    
    async def _handle_it_support(self, user_input: str) -> str:
        """Sequential agent workflow for IT support"""
        # Step 1: Check if it's a common issue
        common_issues = await self.support_agent.check_knowledge_base(user_input)
        
        if common_issues:
            return common_issues
        
        # Step 2: Try to resolve with tools
        tools_response = await self.orchestrator.use_tools(
            user_input, 
            tool_types=["it_tools"]
        )
        
        # Step 3: If unresolved, create ticket
        if "unresolved" in tools_response.lower():
            ticket_id = await self.it_tools[0].create_ticket(
                description=user_input,
                priority="medium"
            )
            return f"I've created a support ticket #{ticket_id} for you."
        
        return tools_response
    
    async def _handle_hr_inquiry(self, user_input: str) -> str:
        """Handle HR inquiries with memory recall"""
        # Check memory bank first
        memory_result = self.memory_bank.recall(
            user_id=self.session_id,
            key="hr_inquiry"
        )
        
        # Use HR tools
        hr_response = await self.orchestrator.use_tools(
            user_input,
            tool_types=["hr_tools"]
        )
        
        if memory_result:
            return f"Based on our previous conversations, I remember you asked about similar topics. {hr_response}"
        
        return hr_response
    
    async def _handle_data_request(self, user_input: str) -> str:
        """Parallel data processing"""
        # Run multiple analyses in parallel
        tasks = [
            self.data_agent.analyze_trends(),
            self.data_agent.generate_report(user_input),
            self.data_agent.calculate_metrics()
        ]
        
        results = await asyncio.gather(*tasks)
        
        return f"Data Analysis Complete:\n1. Trends: {results[0]}\n2. Report: {results[1]}\n3. Metrics: {results[2]}"
    
    async def _handle_workflow(self, user_input: str) -> str:
        """Sequential workflow execution"""
        return await self.workflow_agent.execute_workflow(user_input)
    
    async def _handle_general_support(self, user_input: str) -> str:
        """General support with Google search fallback"""
        response = await self.support_agent.answer_question(user_input)
        
        if "I don't know" in response:
            # Use Google search tool
            search_result = self.google_search.run(f"{user_input} business operations")
            return f"I found this information: {search_result[:200]}..."
        
        return response

# FastAPI deployment
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn

app = FastAPI(title="Business Operations Agent API")

class UserRequest(BaseModel):
    user_id: str
    message: str

@app.post("/process")
async def process_request(request: UserRequest):
    """API endpoint for agent processing"""
    try:
        concierge = BusinessOperationsConcierge(request.user_id)
        result = await concierge.process_request(request.message)
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    return {"status": "healthy", "service": "business-ops-agent"}

import sys
!{sys.executable} -m pip install nest_asyncio -q

import nest_asyncio
nest_asyncio.apply()  # This allows uvicorn to run in notebook

import uvicorn
import asyncio

# Check if we're in a notebook
def is_notebook():
    try:
        from IPython import get_ipython
        return get_ipython() is not None
    except:
        return False

if is_notebook():
    print("📓 Running in Jupyter notebook mode...")
    
    # Method 1: Run server in background thread
    import threading
    
    def run_server():
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        asyncio.run(server.serve())
    
    # Start server
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    
    print("🚀 Server started in background thread")
    print("📡 API available at: http://localhost:8000")
    print("📚 Docs at: http://localhost:8000/docs")
    print("\n🛑 To stop: Kernel → Interrupt")
    
else:
    # Normal execution outside notebook
    print("💻 Running in standard Python mode...")
    uvicorn.run(app, host="0.0.0.0", port=8000)

### 2. Orchestrator Agent (agents/orchestrator.py)

In [ ]:
"""
Orchestrator Agent - Demonstrates A2A Protocol and agent coordination
"""

from typing import List, Dict, Any
from abc import ABC, abstractmethod
import json

class OrchestratorAgent:
    """Main orchestrator implementing A2A protocol patterns"""
    
    def __init__(self, llm, tools, support_agent, workflow_agent, data_agent, memory_bank):
        self.llm = llm
        self.tools = tools
        self.support_agent = support_agent
        self.workflow_agent = workflow_agent
        self.data_agent = data_agent
        self.memory_bank = memory_bank
        self.conversation_history = []
        
    async def use_tools(self, query: str, tool_types: List[str] = None) -> str:
        """Use available tools based on query"""
        
        # Filter tools by type if specified
        available_tools = self.tools
        if tool_types:
            available_tools = [t for t in self.tools 
                             if hasattr(t, 'tool_type') and t.tool_type in tool_types]
        
        # Create prompt for tool selection
        tool_descriptions = "\n".join([f"- {t.name}: {t.description}" 
                                      for t in available_tools])
        
        prompt = f"""
        You have access to these tools:
        {tool_descriptions}
        
        User query: {query}
        
        Choose the most appropriate tool and provide the arguments as JSON.
        Return format: {{"tool": "tool_name", "args": {{"arg1": "value1"}}}}
        """
        
        # Get tool selection from LLM
        response = await self.llm.ainvoke(prompt)
        
        try:
            tool_call = json.loads(response.content)
            tool_name = tool_call["tool"]
            args = tool_call.get("args", {})
            
            # Find and execute the tool
            for tool in available_tools:
                if tool.name == tool_name:
                    result = tool.run(**args)
                    
                    # Log the tool execution
                    self.conversation_history.append({
                        "type": "tool_call",
                        "tool": tool_name,
                        "args": args,
                        "result": str(result)[:200]  # Truncate long results
                    })
                    
                    return result
                    
            return f"Tool {tool_name} not found"
            
        except json.JSONDecodeError:
            return f"Could not parse tool selection: {response.content}"
    
    async def coordinate_agents(self, task: str) -> Dict[str, Any]:
        """Coordinate between multiple agents (A2A Protocol)"""
        
        # Analyze task complexity
        analysis_prompt = f"""
        Analyze this task and determine which agents should handle it:
        Task: {task}
        
        Available agents:
        1. Support Specialist: For questions and information
        2. Workflow Agent: For process execution
        3. Data Agent: For analysis and reporting
        
        Return JSON: {{"primary_agent": "agent_name", "supporting_agents": ["agent1", "agent2"]}}
        """
        
        analysis = await self.llm.ainvoke(analysis_prompt)
        
        try:
            plan = json.loads(analysis.content)
            primary = plan["primary_agent"]
            supporting = plan.get("supporting_agents", [])
            
            results = {"primary": None, "supporting": {}}
            
            # Execute primary agent
            if primary == "support":
                results["primary"] = await self.support_agent.answer_question(task)
            elif primary == "workflow":
                results["primary"] = await self.workflow_agent.execute_workflow(task)
            elif primary == "data":
                results["primary"] = await self.data_agent.analyze_trends(task)
            
            # Execute supporting agents in parallel (simulated)
            for agent_name in supporting:
                if agent_name == "data" and "data" not in [primary] + supporting:
                    results["supporting"]["data"] = await self.data_agent.generate_report(task)
            
            return results
            
        except Exception as e:
            return {"error": str(e)}

### 3. Custom Tools (tools/hr_tools.py)

In [ ]:
"""
Custom HR Tools - Demonstrates custom tools and OpenAPI integration
"""

from typing import Optional, Dict, Any
from datetime import datetime
import requests

class HRToolkit:
    """Collection of HR-related tools"""
    
    def get_tools(self):
        return [
            LeaveBalanceTool(),
            PolicyLookupTool(),
            EmployeeDirectoryTool(),
            BenefitsEnrollmentTool()
        ]

class LeaveBalanceTool:
    """Custom tool for checking leave balance"""
    
    name = "check_leave_balance"
    description = "Check an employee's remaining leave balance"
    tool_type = "hr_tools"
    
    def __init__(self):
        # Simulated database
        self.leave_data = {
            "EMP001": {"vacation": 15, "sick": 10, "personal": 5},
            "EMP002": {"vacation": 20, "sick": 8, "personal": 3}
        }
    
    def run(self, employee_id: str, leave_type: Optional[str] = None) -> str:
        """Execute the tool"""
        if employee_id not in self.leave_data:
            return f"Employee {employee_id} not found"
        
        if leave_type:
            if leave_type in self.leave_data[employee_id]:
                balance = self.leave_data[employee_id][leave_type]
                return f"Employee {employee_id} has {balance} days of {leave_type} leave remaining"
            else:
                return f"Leave type {leave_type} not found"
        
        # Return all balances
        balances = self.leave_data[employee_id]
        return f"Leave balances for {employee_id}: {balances}"

class PolicyLookupTool:
    """Tool to look up HR policies"""
    
    name = "lookup_policy"
    description = "Look up HR policies by topic"
    tool_type = "hr_tools"
    
    def __init__(self):
        self.policies = {
            "remote_work": "Employees may work remotely up to 3 days per week with manager approval.",
            "leave_request": "Submit leave requests at least 2 weeks in advance via the HR portal.",
            "expenses": "Expense reports must be submitted within 30 days with receipts."
        }
    
    def run(self, policy_topic: str) -> str:
        return self.policies.get(
            policy_topic.lower(),
            f"No policy found for topic: {policy_topic}"
        )

class EmployeeDirectoryTool:
    """OpenAPI tool integration example"""
    
    name = "employee_directory"
    description = "Look up employee information"
    tool_type = "hr_tools"
    
    def __init__(self):
        self.api_url = "https://api.example-hr.com/v1/employees"
        # In real implementation, you'd have proper auth headers
    
    def run(self, employee_id: Optional[str] = None, 
            department: Optional[str] = None) -> Dict[str, Any]:
        """Simulated OpenAPI call"""
        # Simulated API response
        return {
            "employees": [
                {"id": "EMP001", "name": "John Doe", "department": "Engineering"},
                {"id": "EMP002", "name": "Jane Smith", "department": "Marketing"}
            ]
        }

class BenefitsEnrollmentTool:
    """Tool for benefits enrollment workflow"""
    
    name = "enroll_benefits"
    description = "Enroll employee in benefits program"
    tool_type = "hr_tools"
    
    def run(self, employee_id: str, benefit_type: str, 
            coverage_level: str = "standard") -> str:
        # Simulate enrollment process
        return f"Enrolled employee {employee_id} in {benefit_type} ({coverage_level} coverage). Confirmation: BEN{datetime.now().strftime('%Y%m%d%H%M%S')}"

### 4. Memory Management (memory/memory_bank.py)

In [ ]:
"""
Memory Bank - Demonstrates long-term memory and context compaction
"""

from typing import Dict, List, Any, Optional
from datetime import datetime, timedelta
import json
from collections import defaultdict

class MemoryBank:
    """Long-term memory storage with context compaction"""
    
    def __init__(self, max_entries_per_user: int = 100):
        self.memory_store = defaultdict(list)
        self.max_entries = max_entries_per_user
        
    def store(self, user_id: str, key: str, value: Any, 
              metadata: Optional[Dict] = None) -> str:
        """Store information in long-term memory"""
        
        entry = {
            "id": f"{user_id}_{key}_{datetime.now().timestamp()}",
            "key": key,
            "value": value,
            "timestamp": datetime.now().isoformat(),
            "metadata": metadata or {},
            "access_count": 0
        }
        
        # Add to user's memory
        self.memory_store[user_id].append(entry)
        
        # Apply context compaction if needed
        if len(self.memory_store[user_id]) > self.max_entries:
            self._compact_context(user_id)
        
        return entry["id"]
    
    def recall(self, user_id: str, key: Optional[str] = None, 
               limit: int = 5) -> List[Dict]:
        """Recall information from memory"""
        
        if user_id not in self.memory_store:
            return []
        
        memories = self.memory_store[user_id]
        
        # Filter by key if specified
        if key:
            relevant = [m for m in memories if key in m["key"]]
        else:
            relevant = memories
        
        # Sort by recency and access count
        relevant.sort(key=lambda x: (
            datetime.fromisoformat(x["timestamp"]), 
            x["access_count"]
        ), reverse=True)
        
        # Update access count
        for memory in relevant[:limit]:
            memory["access_count"] += 1
        
        return relevant[:limit]
    
    def _compact_context(self, user_id: str):
        """Context compaction: remove least accessed memories"""
        memories = self.memory_store[user_id]
        
        # Sort by access count and recency
        memories.sort(key=lambda x: (x["access_count"], 
                                    datetime.fromisoformat(x["timestamp"])))
        
        # Keep only the most recent/accessed memories
        self.memory_store[user_id] = memories[-self.max_entries//2:]
    
    def get_user_patterns(self, user_id: str) -> Dict[str, Any]:
        """Analyze patterns in user interactions"""
        memories = self.memory_store.get(user_id, [])
        
        if not memories:
            return {}
        
        # Analyze frequencies
        key_counts = defaultdict(int)
        for memory in memories:
            key_counts[memory["key"]] += 1
        
        # Find most common topics
        common_topics = sorted(key_counts.items(), 
                             key=lambda x: x[1], 
                             reverse=True)[:3]
        
        return {
            "total_memories": len(memories),
            "common_topics": dict(common_topics),
            "last_accessed": memories[-1]["timestamp"] if memories else None
        }

### 5. Evaluation Suite (evaluation/test_suite.py)

In [ ]:
"""
Agent Evaluation - Demonstrates systematic agent testing
"""

import asyncio
from typing import List, Dict, Tuple
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

class AgentEvaluator:
    """Evaluate agent performance"""
    
    def __init__(self, agent):
        self.agent = agent
        self.test_cases = self._load_test_cases()
        
    def _load_test_cases(self) -> List[Dict]:
        """Load test cases for evaluation"""
        return [
            {
                "input": "What's my vacation balance?",
                "expected_intent": "hr_inquiry",
                "expected_tool": "check_leave_balance"
            },
            {
                "input": "My laptop won't turn on",
                "expected_intent": "it_support",
                "expected_tool": "create_ticket"
            },
            {
                "input": "Show me last month's support metrics",
                "expected_intent": "data_analysis",
                "expected_tool": "generate_report"
            },
            {
                "input": "How do I submit an expense report?",
                "expected_intent": "workflow",
                "expected_tool": "lookup_policy"
            }
        ]
    
    async def evaluate_intent_classification(self) -> Dict[str, float]:
        """Evaluate intent classification accuracy"""
        predictions = []
        true_labels = []
        
        for test_case in self.test_cases:
            # Get agent's intent classification
            result = await self.agent.process_request(test_case["input"])
            predicted_intent = result.get("intent", "unknown")
            
            predictions.append(predicted_intent)
            true_labels.append(test_case["expected_intent"])
        
        # Calculate metrics
        accuracy = accuracy_score(true_labels, predictions)
        
        # Get unique intents for multi-class metrics
        unique_intents = list(set(true_labels + predictions))
        
        precision = precision_score(true_labels, predictions, 
                                  labels=unique_intents, average='weighted', zero_division=0)
        recall = recall_score(true_labels, predictions,
                            labels=unique_intents, average='weighted', zero_division=0)
        
        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "total_tests": len(self.test_cases),
            "detailed_results": list(zip(true_labels, predictions))
        }
    
    async def evaluate_tool_selection(self) -> Dict[str, Any]:
        """Evaluate if correct tools are selected"""
        correct_selections = 0
        details = []
        
        for test_case in self.test_cases:
            # Simulate tool selection (in real scenario, log tool calls)
            result = await self.agent.process_request(test_case["input"])
            
            # Check if expected tool was in the response
            expected_tool = test_case["expected_tool"]
            response_text = str(result.get("response", "")).lower()
            
            # Simple check - in production, you'd parse actual tool calls
            if expected_tool in response_text:
                correct_selections += 1
                details.append({"test": test_case["input"], "correct": True})
            else:
                details.append({"test": test_case["input"], "correct": False})
        
        accuracy = correct_selections / len(self.test_cases)
        
        return {
            "tool_selection_accuracy": accuracy,
            "correct_selections": correct_selections,
            "total_tests": len(self.test_cases),
            "details": details
        }
    
    async def run_complete_evaluation(self) -> Dict[str, Any]:
        """Run all evaluations"""
        print("Starting agent evaluation...")
        
        intent_results = await self.evaluate_intent_classification()
        tool_results = await self.evaluate_tool_selection()
        
        # Calculate overall score
        overall_score = (
            intent_results["accuracy"] * 0.6 +
            tool_results["tool_selection_accuracy"] * 0.4
        )
        
        return {
            "overall_score": overall_score,
            "intent_classification": intent_results,
            "tool_selection": tool_results,
            "passing_threshold": 0.7,
            "passed": overall_score >= 0.7,
            "timestamp": datetime.now().isoformat()
        }

### 6. Deployment Configuration (Dockerfile)

In [ ]:
# ============================================
# CREATE DEPLOYMENT FILES
# ============================================
import os

print("📦 Creating deployment configuration...")

# 1. Create Dockerfile
dockerfile = """# Business Operations Concierge Agent
FROM python:3.11-slim

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application
COPY . .

# Expose port
EXPOSE 8000

# Run application
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile)
print("✅ Created: Dockerfile")

# 2. Create docker-compose.yml (optional)
docker_compose = """version: '3.8'

services:
  business-agent:
    build: .
    ports:
      - "8000:8000"
    environment:
      - OPENAI_API_KEY=${OPENAI_API_KEY}
      - SERPAPI_API_KEY=${SERPAPI_API_KEY}
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "python", "-c", "import requests; requests.get('http://localhost:8000/health')"]
      interval: 30s
      timeout: 10s
      retries: 3
"""

with open("docker-compose.yml", "w") as f:
    f.write(docker_compose)
print("✅ Created: docker-compose.yml")

# 3. Create .dockerignore
dockerignore = """__pycache__
*.pyc
*.pyo
*.pyd
.Python
env
venv
.venv
.env
.git
.gitignore
README.md
*.ipynb
.ipynb_checkpoints
"""

with open(".dockerignore", "w") as f:
    f.write(dockerignore)
print("✅ Created: .dockerignore")

# 4. Create deployment instructions
deploy_readme = """# Deployment Guide

## Quick Start with Docker

```bash
# Build the image
docker build -t business-ops-agent .

# Run the container
docker run -p 8000:8000 -e OPENAI_API_KEY=your_key business-ops-agent

In [ ]:
# Dockerfile for agent deployment
FROM python:3.11-slim

WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \
    gcc \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Create non-root user
RUN useradd -m -u 1000 agentuser && chown -R agentuser:agentuser /app
USER agentuser

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \
    CMD python -c "import requests; requests.get('http://localhost:8000/health')"

# Expose port
EXPOSE 8000

# Run the application
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]

### 7. Requirements File (requirements.txt)

In [ ]:
# Core dependencies
langchain==0.1.0
langchain-openai==0.0.2
langchain-community==0.0.10
openai==1.3.0

# Web framework
fastapi==0.104.1
uvicorn==0.24.0
pydantic==2.5.0

# Tools and utilities
google-search-results==2.4.2
requests==2.31.0
pandas==2.1.3
numpy==1.24.3

# Observability
prometheus-client==0.19.0
structlog==23.2.0

# Testing
pytest==7.4.3
pytest-asyncio==0.21.1

### 8. Running and Testing

In [ ]:
# 1. Install dependencies
pip install -r requirements.txt

# 2. Run the agent system
python app.py

# 3. Test the API (in another terminal)
curl -X POST "http://localhost:8000/process" \
  -H "Content-Type: application/json" \
  -d '{"user_id": "EMP001", "message": "What is my vacation balance?"}'

# 4. Run evaluation
python -m evaluation.test_suite

# 5. Build and deploy with Docker
docker build -t business-ops-agent .
docker run -p 8000:8000 --env OPENAI_API_KEY=your_key business-ops-agent